In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context('poster')
sns.set_style('ticks')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'cmr10'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units

In [ ]:
from importlib import reload
import extract_forcing

In [ ]:
reload(extract_forcing)
from extract_forcing import *

In [ ]:
h = 30. # surface elevation of ENA site in m
lat = 39.0916 # deg N
lon = -28.0257 # deg E
forc_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/forcing'
sonde_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/obs/enasondewnpnC1'

In [ ]:
# fcor = metpy.calc.coriolis_parameter(lat*units('degrees'))
# print(f'{fcor.magnitude:12.5e}')

In [ ]:
t1 = '2018-02-07T00:00:00'
t2 = '2018-02-08T00:00:00'

In [ ]:
dz = 25. # m
zout = np.arange(dz*0.5, 5000.+ dz*0.6, dz)

In [ ]:
dx = 0.75 # degrees

In [ ]:
z_snd, tp_snd, rv_snd, u_snd, v_snd = extract_sounding_sonde(f'{sonde_dir}/enasondewnpnC1.b1.20180207.053000.cdf')

In [ ]:
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-sfc.nc')
era5_l = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-ml.nc')
era5_sfc = era5_sfc.sortby(era5_sfc.time)
toy_era5_sfc, sst_era5, shf_era5, lhf_era5, _ = extract_sfc_fluxes_era5(era5_sfc, t1, t2, lat, lon, dx)
toy_era5, zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg = extract_forcing_era5(era5_l, t1, t2, lat, lon, dx)
w_era5, u_era5, v_era5, ta_era5, ma_era5, ug_era5, vg_era5 = interpolate_forcing_era5(zout, z, w, u, v, ta, ma, ug, vg, zs)
ps_era5 = ps.mean(axis=(1,2))
# del zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg
del era5_l, era5_sfc

In [ ]:
"""
The formats of input files for SAM
- snd
 z[m] p[mb] tp[K] q[g/kg] u[m/s] v[m/s]
 0.,  6, 1015.  day,levels,pres0
0.	-999.   298.70	17.00	-8.75	0.0
- lsf
 z[m] p[mb]  tls[K/s] qls[kg/kg/s] uls  vls  wls[m/s]
 0.,  6   1015. day, nlevels, pres0
 0.	-999. -2.315e-05  -1.2e-08	-10.00	0.	0.
- sfc
 day sst(K) H(W/m2) LE(W/m2) TAU(m2/s2)
 0.	301.   9.46   153.4	0.0784
"""
t_start = 37.25
t_end = 37.875
ztop = 5500.0

nt_start = np.nonzero(toy_era5 == t_start)[0][0]
nt_end = np.nonzero(toy_era5 == t_end)[0][0]
lim = z_snd < 5500.0
z_snd_out = np.compress(lim, z_snd)
tp_snd_out = np.compress(lim, tp_snd)
rv_snd_out = np.compress(lim, rv_snd)
rv_snd_out = rv_snd_out * 1.0e3  # convert to g/kg
u_snd_out = np.compress(lim, u_snd)
v_snd_out = np.compress(lim, v_snd)
nz = z_snd_out.shape[0]
with open("snd_era5ps", "w") as snd:
    snd.write("z[m] p[mb] tp[K] q[g/kg] u[m/s] v[m/s]\n")
    snd.write(
        f"{t_start},  {nz},  {ps_era5.values[nt_start]/100.0:8.3f}  day,levels,pres0\n"
    )
    for z, t, r, u, v in zip(
        z_snd_out, tp_snd_out.magnitude, rv_snd_out.magnitude, u_snd_out, v_snd_out
    ):
        snd.write(f"{z - h:8.2f}  -999.9  {t:8.4f}  {r:8.4f}  {u:9.4f}  {v:9.4f} \n")
    snd.write(
        f"{t_end},  {nz},  {ps_era5.values[nt_start]/100.0:8.3f}  day,levels,pres0\n"
    )
    for z, t, r, u, v in zip(
        z_snd_out, tp_snd_out.magnitude, rv_snd_out.magnitude, u_snd_out, v_snd_out
    ):
        snd.write(f"{z - h:8.2f}  -999.9  {t:8.4f}  {r:8.4f}  {u:9.4f}  {v:9.4f} \n")

nt_start = np.nonzero(toy_era5_sfc == t_start)[0][0]
nt_end = np.nonzero(toy_era5_sfc == t_end)[0][0]
with open("sfc_era5", "w") as sfc:
    sfc.write("day sst(K) H(W/m2) LE(W/m2) TAU(m2/s2) \n")
    for nt in range(nt_start, nt_end + 1):
        sfc.write(
            f"{toy_era5_sfc[nt]:13.9f}  {sst_era5[nt]:8.4f}  {-shf_era5[nt]:9.4f}  {-lhf_era5.values[nt]:9.4f}  -999.9 \n"
        )

nt_start = np.nonzero(toy_era5 == t_start)[0][0]
nt_end = np.nonzero(toy_era5 == t_end)[0][0]
nz = zout.shape[0]
with open("lsf_era5", "w") as lsf:
    lsf.write(" z[m] p[mb]  tls[K/s] qls[kg/kg/s] uls  vls  wls[m/s] \n")
    for nt in range(nt_start, nt_end + 1):
        lsf.write(
            f"{toy_era5[nt]:13.9f},  {nz},  {ps_era5.values[nt]/100.0:8.3f}  day,levels,pres0 \n"
        )
        for zl in range(nz):
            lsf.write(
                f"{zout[zl]:9.2f}  -999.9  {ta_era5[nt, zl]:12.4e}  {ma_era5[nt, zl]:12.4e}  {ug_era5[nt, zl]:9.4f}  {vg_era5[nt, zl]:9.4f}  {w_era5[nt, zl]:12.4e} \n"
            )

In [ ]:
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.20180207.nc4')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180207.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180207.nc4')
# merra2_sfc0 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_101.const_2d_asm_Nx.00000000.nc4')
toy_merra2_sfc, sst_merra2, shf_merra2, lhf_merra2 = extract_sfc_fluxes_merra2(merra2_sfc, merra2_sfc2, t1, t2, lat, lon, dx)
toy_merra2, zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg = extract_forcing_merra2(merra2_l, t1, t2, lat, lon, dx)
w_merra2, u_merra2, v_merra2, ta_merra2, ma_merra2, ug_merra2, vg_merra2 = interpolate_forcing_merra2(zout, z, w, u, v, ta, ma, ug, vg, zs)
ps_merra2 = ps.mean(axis=(1,2))
del zs, ps, z, p, w, omega, u, v, ta, ma, ug, vg
del merra2_l, merra2_sfc, merra2_sfc2

In [ ]:
"""
The formats of input files for SAM
- snd
 z[m] p[mb] tp[K] q[g/kg] u[m/s] v[m/s]
 0.,  6, 1015.  day,levels,pres0
0.	-999.   298.70	17.00	-8.75	0.0
- lsf
 z[m] p[mb]  tls[K/s] qls[kg/kg/s] uls  vls  wls[m/s]
 0.,  6   1015. day, nlevels, pres0
 0.	-999. -2.315e-05  -1.2e-08	-10.00	0.	0.
- sfc
 day sst(K) H(W/m2) LE(W/m2) TAU(m2/s2)
 0.	301.   9.46   153.4	0.0784
"""
t_start = 37.25
t_end = 37.875
ztop = 5500.0

nt_start = np.nonzero(toy_merra2 == t_start)[0][0]
nt_end = np.nonzero(toy_merra2 == t_end)[0][0]
print(toy_merra2[nt_start])
print(toy_merra2[nt_end])
lim = z_snd < 5500.0
z_snd_out = np.compress(lim, z_snd)
tp_snd_out = np.compress(lim, tp_snd)
rv_snd_out = np.compress(lim, rv_snd)
rv_snd_out = rv_snd_out * 1.0e3  # convert to g/kg
u_snd_out = np.compress(lim, u_snd)
v_snd_out = np.compress(lim, v_snd)
nz = z_snd_out.shape[0]
with open("snd_merra2ps", "w") as snd:
    snd.write("z[m] p[mb] tp[K] q[g/kg] u[m/s] v[m/s]\n")
    snd.write(
        f"{t_start},  {nz},  {ps_merra2.values[nt_start]/100.0:8.3f}  day,levels,pres0\n"
    )
    for z, t, r, u, v in zip(
        z_snd_out, tp_snd_out.magnitude, rv_snd_out.magnitude, u_snd_out, v_snd_out
    ):
        snd.write(f"{z - h:8.2f}  -999.9  {t:8.4f}  {r:8.4f}  {u:9.4f}  {v:9.4f} \n")
    snd.write(
        f"{t_end},  {nz},  {ps_merra2.values[nt_start]/100.0:8.3f}  day,levels,pres0\n"
    )
    for z, t, r, u, v in zip(
        z_snd_out, tp_snd_out.magnitude, rv_snd_out.magnitude, u_snd_out, v_snd_out
    ):
        snd.write(f"{z - h:8.2f}  -999.9  {t:8.4f}  {r:8.4f}  {u:9.4f}  {v:9.4f} \n")

nt_start = np.nonzero(toy_merra2_sfc >= t_start)[0][0] - 1
nt_end = np.nonzero(toy_merra2_sfc <= t_end)[0][-1] + 1
print(toy_merra2_sfc[nt_start])
print(toy_merra2_sfc[nt_end])
with open("sfc_merra2", "w") as sfc:
    sfc.write("day sst(K) H(W/m2) LE(W/m2) TAU(m2/s2) \n")
    for nt in range(nt_start, nt_end + 1):
        sfc.write(
            f"{toy_merra2_sfc[nt]:13.9f}  {sst_merra2[nt]:8.4f}  {shf_merra2[nt]:9.4f}  {lhf_merra2.values[nt]:9.4f}  -999.9 \n"
        )

nt_start = np.nonzero(toy_merra2 == t_start)[0][0]
nt_end = np.nonzero(toy_merra2 == t_end)[0][0]
print(toy_merra2[nt_start])
print(toy_merra2[nt_end])
nz = zout.shape[0]
with open("lsf_merra2", "w") as lsf:
    lsf.write(" z[m] p[mb]  tls[K/s] qls[kg/kg/s] uls  vls  wls[m/s] \n")
    for nt in range(nt_start, nt_end + 1):
        lsf.write(
            f"{toy_merra2[nt]:13.9f},  {nz},  {ps_merra2.values[nt]/100.0:8.3f}  day,levels,pres0 \n"
        )
        for zl in range(nz):
            lsf.write(
                f"{zout[zl]:9.2f}  -999.9  {ta_merra2[nt, zl]:12.4e}  {ma_merra2[nt, zl]:12.4e}  {ug_merra2[nt, zl]:9.4f}  {vg_merra2[nt, zl]:9.4f}  {w_merra2[nt, zl]:12.4e} \n"
            )

In [ ]:
tstamp = '2008-02-07T06:00:00'
toy = 37.25
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
lim = z_snd < 5500.
zlim = np.compress(lim, z_snd)
u_snd_out = np.compress(lim, u_snd)
v_snd_out = np.compress(lim, v_snd)
nt = np.nonzero(toy_merra2==toy)[0][0]
# print(toy_merra2[nt])
ax1.plot(u_snd_out, zlim*1.0e-3, color='red', label='u - SONDE')
ax2.plot(v_snd_out, zlim*1.0e-3, color='red', label='v - SONDE')
ax1.plot(u_merra2[nt,:], zout*1.0e-3, color='black', label='u - MERRA2')
ax1.plot(ug_merra2[nt,:], zout*1.0e-3, color='black', label='ug - MERRA2', linestyle='--')
ax2.plot(v_merra2[nt,:], zout*1.0e-3, color='black', label='v - MERRA2')
ax2.plot(vg_merra2[nt,:], zout*1.0e-3, color='black', label='vg - MERRA2', linestyle='--')
nt = np.nonzero(toy_era5==toy)[0][0]
# print(toy_era5[nt])
ax1.plot(u_era5[nt,:], zout*1.0e-3, color='blue', label='u - ERA5')
ax1.plot(ug_era5[nt,:], zout*1.0e-3, color='blue', label='ug - ERA5', linestyle='--')
ax2.plot(v_era5[nt,:], zout*1.0e-3, color='blue', label='v - ERA5')
ax2.plot(vg_era5[nt,:], zout*1.0e-3, color='blue', label='vg - ERA5', linestyle='--')
ax1.set_ylim(0, 5)
# ax1.set_xlim(-10, 0)
ax1.set_ylabel('Altitude (km)')
ax1.set_xlabel(r' $u$ (m/s)')
ax1.set_title(tstamp)
ax1.legend(fontsize=12)
ax2.set_ylim(0, 5)
ax2.set_ylabel('')
ax2.set_xlabel(r' $v$ (m/s)')
ax2.legend(fontsize=12)
ax2.set_title(tstamp)
plt.show()